# Imports and Constants

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
import random
import os

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *
from utility import *

In [4]:
maps_folder="brain_maps"
masks_folder="masks"

SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_perms = 1000
class_labels = [""] * (2 * length_one_modality)
for i in range(length_one_modality * 2):
    class_labels[i] = classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length_one_modality])
labels["aud"] = np.array(class_labels[length_one_modality:])
labels_same = labels["vis"]

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                           (["vis"], ["PT_L", "PT_R"]),
                           (["aud"], ["V5_L", "V5_R"]),
                           (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                            (["vis", "aud"], ["PT_L", "PT_R"])]

scaler = sklearn.preprocessing.StandardScaler()
classifier = sklearn.svm.SVC(C = 1, kernel = 'linear', random_state=SEED)  # other parameters are default parameters
pipeline = Pipeline([('std_scaler', scaler), ('svm', classifier)])
decoder = Decoder(n_perm=n_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)
_ = decoder.return_and_reset_predictions(classes)

# Loading data

In [5]:
maps_masked = load_full_data(subjects_ids, length_one_modality, maps_folder, masks_folder)

# Within-modality decoding

In [8]:
within_cv_scores = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
within_modality_group_results = average_dicos(within_cv_scores)

# Cross-modal decoding

In [9]:
cross_cv_scores = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
cross_modality_group_results = average_dicos(cross_cv_scores)

# Bootstrapping to assess group-level significance

In [7]:
n_single_perm = 100
n_bootstrap = 100_000

In [ ]:
within_scores_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same,within_modal_tasks_regions,maps_masked,n_subjects, within_modality=True)
bootstrapped_distribution_within = compute_bootstrap_distribution(n_bootstrap, n_subjects, within_scores_100_perm, n_single_perm)

In [ ]:
cross_scores_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same,cross_modal_tasks_regions,maps_masked,n_subjects, within_modality=False)
bootstrapped_distribution_cross = compute_bootstrap_distribution(n_bootstrap, n_subjects, cross_scores_100_perm, n_single_perm)

# Saving results

In [ ]:
out_directory = "out/"+maps_folder+"/"+masks_folder+"/"
create_directory(out_directory)

In [ ]:
save_dicts("within_cv_scores.csv", within_cv_scores, list(within_cv_scores[0].keys()), subjects_ids)
save_dicts("group_scores_within.csv", [within_modality_group_results], list(within_modality_group_results.keys()), [0])

In [ ]:
save_dicts("cross_cv_scores.csv", cross_cv_scores, list(cross_cv_scores[0].keys()), subjects_ids)
save_dicts("group_scores_cross.csv", [cross_modality_group_results], list(cross_modality_group_results.keys()), [0])

In [12]:
save_dicts("bootstraps_within.csv", [bootstrapped_distribution_within], list(bootstrapped_distribution_within[0].keys()), range(n_bootstrap))

In [ ]:
save_dicts("bootstraps_cross.csv", [bootstrapped_distribution_cross], list(bootstrapped_distribution_cross[0].keys()), range(n_bootstrap))

# Plotting results (from files of saved results)

In [4]:
# joining dataframes in prevision of plotting
cv_within_df = pd.read_csv("out/group_scores_within.csv", index_col=0)
cv_cross_df = pd.read_csv("out/group_scores_cross.csv", index_col=0)
for col in cv_cross_df.columns: cv_within_df[col] = cv_cross_df[col]

bootstrap_within_df = pd.read_csv("out/bootstraps_within.csv", index_col=0)
bootstrap_cross_df = pd.read_csv("out/bootstraps_cross.csv", index_col=0)
for col in bootstrap_cross_df.columns: bootstrap_within_df[col] = bootstrap_cross_df[col]

# compute estimated p-values for bootstrap
pvals = compute_p_val_bootstrap(bootstrap_within_df, cv_within_df)
save_dicts("estimated_pval_bootstrap.csv", [pvals], list(pvals.keys()), [0])

#perm_df = pd.read_csv("out/scores_perms.csv", index_col=0)

In [5]:
plt_directory = "plots"+maps_folder+"/"+masks_folder+"/"
create_directory(plt_directory)

plotter = Plotter(plt_directory, subjects_ids)
plotter.plot_cv_score(cv_within_df, chance_level = True)
plotter.plot_bootstrap(bootstrap_within_df, cv_within_df, pvals, 30)
#plotter.plot_perms_scores(perm_df, n_perms)